<a href="https://colab.research.google.com/github/jdu8/sentiment-analysis-comparison/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Sun Nov  6 10:38:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d kazanova/sentiment140

mkdir: cannot create directory ‘/root/.kaggle’: File exists
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
! unzip sentiment140.zip

Archive:  sentiment140.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input,Dropout,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding,Bidirectional,LSTM
import keras

In [5]:
df=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',header=None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
columns=df.columns
columns

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [7]:
df.drop([1,2,3,4],axis=1,inplace=True)
df.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
df.columns=['sentiment','data']
df.head()

,sentiment,data
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [9]:
y=df['sentiment']

In [10]:
df_train,df_test,y_train,y_test=train_test_split(df['data'],y,test_size=0.3)
print('DF Train Shape: ',df_train.shape)
print('DF Test Shape: ',df_test.shape)
print('Y Train Shape: ',y_train.shape)
print('Y Test Shape: ',y_test.shape)

DF Train Shape:  (1120000,)
DF Test Shape:  (480000,)
Y Train Shape:  (1120000,)
Y Test Shape:  (480000,)


In [11]:
max_words=10000
tokenizer=Tokenizer(max_words)
tokenizer.fit_on_texts(df_train)
sequence_train=tokenizer.texts_to_sequences(df_train)
sequence_test=tokenizer.texts_to_sequences(df_test)

In [12]:
word2vec=tokenizer.word_index
V=len(word2vec)
print(f'Number of independent tokens in dataset: {V}')

Number of independent tokens in dataset: 543878


In [13]:
data_train=pad_sequences(sequence_train)
data_train.shape

(1120000, 45)

In [14]:
T=data_train.shape[1]
data_test=pad_sequences(sequence_test,maxlen=T)
data_test.shape

(480000, 45)

In [15]:
D=50

In [16]:
dnn_model=keras.Sequential([
    Input((T,)),
    Embedding(V+1,D),
    Dense(256,activation='relu'),
    Dropout(0.4),
    Dense(64,activation='relu'),
    Dropout(0.4),
    Dense(5,activation='softmax')
])
dnn_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
dnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 45, 50)            27193950  
                                                                 
 dense (Dense)               (None, 45, 256)           13056     
                                                                 
 dropout (Dropout)           (None, 45, 256)           0         
                                                                 
 dense_1 (Dense)             (None, 45, 64)            16448     
                                                                 
 dropout_1 (Dropout)         (None, 45, 64)            0         
                                                                 
 dense_2 (Dense)             (None, 45, 5)             325       
                                                                 
Total params: 27,223,779
Trainable params: 27,223,779
No

In [17]:
cnn_model=keras.Sequential([
    Input((T,)),
    Embedding(V+1,D),
    Conv1D(32,3,activation='relu'),
    MaxPooling1D(3),
    Conv1D(64,3,activation='relu'),
    MaxPooling1D(3),
    Conv1D(128,3,activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.4),
    Dense(5,activation='softmax')
])
cnn_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 45, 50)            27193950  
                                                                 
 conv1d (Conv1D)             (None, 43, 32)            4832      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 12, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 4, 64)            0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 2, 128)           

In [18]:
rnn_model=keras.Sequential([
    Input((T,)),
    Embedding(V+1,D),
    Bidirectional(LSTM(32)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(5,activation='softmax')
])
rnn_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 45, 50)            27193950  
                                                                 
 bidirectional (Bidirectiona  (None, 64)               21248     
 l)                                                              
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                      

In [ ]:
# dnn_hist=dnn_model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=5,batch_size=100)

In [20]:
cnn_hist=cnn_model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=5,batch_size=100)

Epoch 1/5
11200/11200 [==============================] - 213s 19ms/step - loss: 0.4768 - accuracy: 0.7727 - val_loss: 0.4458 - val_accuracy: 0.7903
Epoch 2/5
11200/11200 [==============================] - 210s 19ms/step - loss: 0.4344 - accuracy: 0.7993 - val_loss: 0.4400 - val_accuracy: 0.7945
Epoch 3/5
11200/11200 [==============================] - 209s 19ms/step - loss: 0.4135 - accuracy: 0.8115 - val_loss: 0.4417 - val_accuracy: 0.7933
Epoch 4/5
11200/11200 [==============================] - 211s 19ms/step - loss: 0.3955 - accuracy: 0.8218 - val_loss: 0.4465 - val_accuracy: 0.7919
Epoch 5/5
11200/11200 [==============================] - 209s 19ms/step - loss: 0.3793 - accuracy: 0.8307 - val_loss: 0.4566 - val_accuracy: 0.7911


In [21]:
rnn_hist=rnn_model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=5,batch_size=100)

Epoch 1/5
11200/11200 [==============================] - 256s 23ms/step - loss: 0.4369 - accuracy: 0.7997 - val_loss: 0.4012 - val_accuracy: 0.8171
Epoch 2/5
11200/11200 [==============================] - 251s 22ms/step - loss: 0.3963 - accuracy: 0.8226 - val_loss: 0.3956 - val_accuracy: 0.8214
Epoch 3/5
11200/11200 [==============================] - 252s 22ms/step - loss: 0.3814 - accuracy: 0.8308 - val_loss: 0.3891 - val_accuracy: 0.8243
Epoch 4/5
11200/11200 [==============================] - 252s 22ms/step - loss: 0.3692 - accuracy: 0.8368 - val_loss: 0.3927 - val_accuracy: 0.8242
Epoch 5/5
11200/11200 [==============================] - 256s 23ms/step - loss: 0.3587 - accuracy: 0.8424 - val_loss: 0.3926 - val_accuracy: 0.8238
